# MemoryBanditWorkflow を使った RAG の試験実装

(Version: 0.0.2)

先に MemoryBanditWorkflow というフレームワークを作った。それは迷路問題の一部として作り始めたが、汎用なものになるようこころがけた。それが実際汎用に使えることを示すために、RAG エージェントを作ってみようというのが今回の試みである。

アイデアの軌跡は↓にある。

\[cocolog:95619779\](2025年9月)  
《「LLM のメモリ機能を強制的に使うバンディットマシンの試験実装」と「LLM のメモリ機能とバンディット機能の試験実装」を行った。後者がメインの成果物で、メモリ機能の使用増加をどう強制するかから拡張したフレームワーク。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2025/09/post-8225e2.html

基本的にはメモリ機能とバンディット機能とワークフロー機能を持っているのが MemoryBanditWorkflow で、今回の物はそれをそのまま使って、RagAgent という子クラスを定義している。

なお、MemoryBanditWorkflow はセマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

MemoryBanditWorkflow の元となった迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、RAG エージェントといえばマルチエージェントを使うのが流行りのようだが、今回の物はリニアに順次、シングルスレッドで実行されていくだけである。マルチエージェントを敷衍するなら asyncio を使ったり、メモリにデータベースを使ったりすべきなのだが、そういうことはできる展望すらない。あくまで MemoryBanditWorkflow で RAG も実装できることを示そうとしたものなので、RAG としての新規性はない。


## 前回へのリンク

《experimental_rag_0.0.1.ipynb - GitHub Gist》  
https://gist.github.com/JRF-2018/6d7dda3eaf7743b91bca32037574bfc1

## 前回からの変更点

MemoryBanditWorkflow に変更はない。

0.0.1 からは RAG のループを少し変えて、一章ごと取材・執筆をするように指示を出し、バンディットが呼ばれやすくならないか試してみた。

## 結論

Gemini 2.5 Pro さんというコストのかかる遅いモデルを使った。やはり頭がいい者にやらせたかったから。前回は修正しながら load して resume をやったが、今回は、ほぼ一発出し。タイトルの設定だけお忘れになったが、他はだいたい論文として仕上がってると思う。

一章ずつというわけにはいかなかったが、おおむね細かい単位で取材してくれて、バンディットが強制されやすくはなっていたと思う。前回に引き続き MemoryBanditWorkflow の実証実験としてはそこそこうまくいったと思う。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。コストもかかるしね \(^^;。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )


## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さんや Grok さん)に教わって作っています。

## 実装

まず、必要なライブラリを読み込みます。

In [1]:
!pip install -q -U langchain langgraph langchain-google-genai langmem duckduckgo-search langchain-community beautifulsoup4 ddgs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.4 

Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata

llm = ChatGoogleGenerativeAI(google_api_key=userdata.get('GOOGLE_API_KEY'), model="models/gemini-2.5-pro")
emb_llm = GoogleGenerativeAIEmbeddings(google_api_key=userdata.get('GOOGLE_API_KEY'), model='gemini-embedding-001')

ちゃんと Gemini にアクセスできるかテストします。

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="以下の質問に回答してください：{question}", input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"question": "Geminiモデルの特徴を教えてください"})
print(response)

/tmp/ipython-input-1709569937.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-1709569937.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": "Geminiモデルの特徴を教えてください"})


はい、承知いたしました。Googleが開発したAIモデル「Gemini」の特徴について、分かりやすく解説します。

Geminiの最大の特徴は、一言で言うと**「生まれながらのマルチモーダルAI」**であることです。これは、AIがテキストだけでなく、画像、音声、動画、コードといった様々な種類の情報を、人間のように自然に理解し、組み合わせて処理できることを意味します。

以下に、Geminiの主な特徴を具体的に解説します。

---

### 1. ネイティブなマルチモーダル (Native Multimodality)

これがGeminiを最も特徴づける点です。従来のAIモデルの多くは、まずテキストを扱うモデルを開発し、後から画像や音声を認識する機能を追加していました。

一方、Geminiは**開発の初期段階から、テキスト、画像、音声、動画などを同時に扱えるように設計**されています。これにより、以下のようなことが可能になります。

*   **情報の統合的な理解：** 例えば、料理をしている動画を見せながら「ここで使っているスパイスは何？」と質問すると、映像と音声を統合的に理解して回答できます。
*   **高度なインタラクション：** ホワイトボードに手書きした物理の問題の図と数式を写真で見せると、それを理解して解法をステップバイステップで説明してくれます。
*   **多様な出力：** 「新しいアプリのUIデザインを描いて」と手書きのスケッチを見せると、それに基づいたHTMLやCSSのコードを生成することができます。

### 2. 卓越した推論能力 (Exceptional Reasoning Ability)

Geminiは、複雑な情報を読み解き、論理的に考える「推論能力」が非常に高いとされています。

*   **専門分野での活用：** 科学論文や膨大な法律文書を読み込ませ、その要点や矛盾点を指摘させるといった高度なタスクを実行できます。
*   **多段階の思考：** 複数のステップを踏まなければ解けない数学の難問や、複雑なロジックパズルを解く能力に長けています。実際に、大学レベルの数学や物理学などの知識を問うベンチマーク（MMLU）で、人間の専門家を上回るスコアを記録した初のモデルとなりました。

### 3. 高度なコーディング能力

Gem

埋め込みベクトルも試しておきます。

In [4]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.013715926557779312,
 -0.0006048799259588122,
 0.011018728837370872,
 -0.09579180181026459,
 0.006502930540591478]

基本的なモジュールを読み込みます。

In [5]:
import os
import math
import numpy as np
import random
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [6]:
RAG_AGENT_SAVE = "rag-agent.pickle"

ライブラリ集。

In [7]:
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy
from IPython.display import Markdown

# LangGraphのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import WebBaseLoader

INITIAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        ("user", "上記の会話の要約を作成してください:"),
    ]
)

EXISTING_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        (
            "user",
            "これまでの会話の要約です: {existing_summary}\n\n"
            "上記の新しいメッセージを考慮して、この要約を拡張してください:",
        ),
    ]
)

FINAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        # if exists
        ("placeholder", "{system_message}"),
        ("system", "これまでの会話の要約: {summary}"),
        ("placeholder", "{messages}"),
    ]
)


In [8]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance


def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str


MemoryBanditWorkflow は変わっていないです。長大な定義で申し訳ないです。

In [9]:
class MemoryBanditWorkflow:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file

        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = []

        self.access_unit = 1.0
        self.recent_reads = []

        self.workflows = {}
        self.workflow_current = "workflow:main"
        self.workflow_next = None
        self.privileged_tool_names = []

        self.init_memories()
        self.init_workflows()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?memory:(\d+)\]?', id_or_num)
            if m:
                return f"memory:{m.group(1)}"
            if id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _normalize_workflow_id_sub(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"workflow:{id_or_num}"
        if id_or_num in ["current", "main"]:
            return f"workflow:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?workflow:(\d+|main|current)\]?(?:.+)?', id_or_num.strip())
            if m:
                return f"workflow:{m.group(1)}"
            if id_or_num.isdigit():
                return f"workflow:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def normalize_workflow_id(self, id_or_num):
        r = self._normalize_workflow_id_sub(id_or_num)
        if r == "workflow:current":
            return self.workflow_current
        return r

    def _replace_tools (self, from_tools, to_tools):
        tool_names = [x.name for x in to_tools]
        return [x for x in from_tools
                if x.name not in tool_names] + to_tools

    def _create_tools (self):
        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"ツール(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent(top_n: int = 10) -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > top_n:
                    sorted_memories = sorted_memories[:top_n]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_list_random(top_n: int = 10) -> Dict[str, Any]:
            """
            メモリをランダムにリストします。
            """

            keys = list(self.memories.keys())
            if len(keys) > top_n:
                keys = random.sample(keys, top_n)
            if keys:
                values = [self.memories[k] for k in keys]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in values]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: メモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "", workflow_id: str = "workflow:current") -> str:
            """
            ツールの利用を強制するためのバンディットを予約します。

            Args:
                tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                aux_prompt: 実行の詳細を示すための補助プロンプト または ""
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            tool_names = re.split(r"\s+or\s+|\s+OR\s+", tool_name)
            prohibited = set(self.privileged_tool_names) & set(tool_names)
            if prohibited:
                return f"失敗。{repr(prohibited)} は登録できません。"
            all_tools = [x.name
                         for x in self._create_tools()['all_tools']]
            if not any (x in all_tools for x in tool_names):
                return f"失敗。{tool_name} は有効なツールでありません。"

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには tool_name exec_mode aux_prompt のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'{tool_name}' は書き換えられません。"
            else:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            memory_read ツールの利用を強制するためのバンディットを予約します。

            Args:
                memory_id: memory_read するための memory_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode memory_id のすべてを一致させて指定する必要があります。"
                x = {'pin': None, 'arg': memory_id}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'memory_read {memory_id}' は書き換えられません。"
            else:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_read): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_workflow(workflow_id_to_schedule: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            workflow_do ツールの利用を強制するためのバンディットを予約します。

            Args:
                workflow_id_to_schedule: workflow_do するための workflow_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            workflow_id2 = self.normalize_workflow_id(workflow_id_to_schedule)
            if workflow_id2 not in self.workflows:
                return f"失敗。{workflow_id2} は有効なワークフローではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "workflow_do" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == workflow_id2:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode workflow_id_to_schedule のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': workflow_id2
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'workflow_do {workflow_id2}' は書き換えられません。"
            else:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_workflow): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_list(workflow_id: str =  "workflow:current")  -> Dict[str, Any]:
            """
            登録されているバンディットのスタックを返します。

            Args:
                workflow_id: バンディットのスタックが登録されたワークフローを指定します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            return {'status': 'success',
                    'result': self.workflows[workflow_id]['stack']}

        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが完了したことを宣言します。サブワークが完了したときのみ使ってください。
            """

            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def workflow_do(workflow_id: str)  -> str:
            """
            ワークフローを実行します。
            """

            self.workflow_next = None
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'e' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は子として実行できません。"
            self.workflow_next = workflow_id
            title = self.workflows[workflow_id]['title']
            return f"成功。この後、{workflow_id} 「{title}」を実行していきます。"

        @tool
        def workflow_list()  -> Dict[str, Any]:
            """
            登録されているすべてのワークフローの id とその title および pin 状況を返します。
            """

            return {'status': 'success',
                    'result': list(self.workflows.values())}

        @tool
        def workflow_show_current()  -> str:
            """
            現在実行中のワークフローのタイトル、および、強制しているバンディットの情報を表示します。
            """

            wtitle = self.workflows[self.workflow_current]['title']
            mes = dedent(f"""\
            現在のワークフロー: {self.workflow_current} 「{wtitle}」
            現在のバンディットの指示: 「{self.cur_bandit_prompt}」
            現在のバンディット: {repr(self.cur_bandit)}
            現在のバンディットの done 数: {self.cur_bandit_done}
            """)
            print(f"ツール(workflow_show_current): {mes}")
            return mes


        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: str)  -> str:
            """
            新しいワークフローを定義し、その workflow_id を返します。

            Args:
                title: ワークフローの名前。
                bandits: 登録するバンディットのリスト。
                pin: 'w' は書換不能。'd' は削除不能。'wd' は書換＆削除不能。'' は書換＆削除可能。

            バンディットはそれぞれ辞書型、それを b とすると。
            b['tool_name'] は bandit_schedule での tool_name と同じ。
            b['exec_mode'] は bandit_schedule の exec_mode と同じ。
            b['aux_prompt'] は bandit_schedule の aux_prompt と同じ。
            b['prob'] は bandit_schedule の prob と同じ。
            b['times'] は bandit_schedule の times と同じ。
            b['arg'] は b['tool_name'] が 'memory_read' のとき memory:〜 という memory_id を指定できる。または、b['tool_name'] が 'workflow_do' のとき workflow:〜 という workflow_id を指定できる。
            b['pin'] は、None のとき削除または prob と times の書き換え可能。'stack' のときは prob と times の書き換え可能。'write' のときは書き換えも不能。
            """
            # AI に対する隠し属性として pin の 'e' は子として実行不能。

            for b in bandits:
                if not all(x in b for x in ['tool_name', 'exec_mode',
                                            'aux_prompt', 'times', 'prob']):
                    return f"失敗。有効な定義ではありません。"
                if 'arg' not in b:
                    b['arg'] = None
                if b['tool_name'] not in ["memory_read", "workflow_do"] \
                   and b['arg']:
                    return f"失敗。有効な定義ではありません。"
                if b['arg'] and b['tool_name'] == "memory_read":
                    b['tools_name'] = "read_tools"
                else:
                    b['tools_name'] = "default_tools"
                if 'pin' not in b:
                    b['pin'] = None
                if not (b['pin'] is None or b['pin'] == 'stack'
                        or b['pin'] == 'write'):
                    return f"失敗。不正な pin 値です。"
                tool_names = re.split(r"\s+or\s+|\s+OR\s+", b['tool_name'])
                prohibited = set(self.privileged_tool_names) & set(tool_names)
                if prohibited:
                    return f"失敗。{repr(prohibited)} は登録できません。"
                all_tools = [x.name
                             for x in self._create_tools()[b['tools_name']]]
                if not any (x in all_tools for x in tool_names):
                    return f"失敗。{b['tool_name']} は有効なツールでありません。"

            i = 1000
            while True:

                if f"workflow:{i}" not in self.workflows:
                    break
                i = i + 1
            new_id = f"workflow:{i}"

            self.workflows[new_id] = {'stack': bandits, 'pin': pin,
                                      'title': title}
            print(f"ツール(workflow_new): {repr(self.workflows[new_id])}")
            return f"成功。{new_id} を新規登録しました。"

        @tool
        def workflow_delete(workflow_id: str)  -> str:
            """
            ワークフローを削除します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} はありません。"
            if 'd' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は削除できません。"
            del self.workflows[workflow_id]
            return f"成功。{workflow_id} を削除しました。"

        r = {}
        r['null_tools'] = []
        r['read_tools'] = r['null_tools'] + [
            express_thought,
            update_plan, show_plan, update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent, memory_list_random,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            bandit_schedule, bandit_schedule_read, bandit_list,
            bandit_statistics,
            subwork_done,
            workflow_do, workflow_new, workflow_list,
            workflow_show_current, workflow_delete,
            bandit_schedule_workflow
        ]
        r['default_tools'] = r['read_tools'] + [
            memory_new, memory_update_string, memory_append_string,
            memory_delete
        ]
        r['all_tools'] = r['default_tools']
        return r

    def _create_agent (self, tools_name='default_tools'):
        tools = self._create_tools()[tools_name]
        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm, tools, prompt=self.system_prompt,
            checkpointer=InMemorySaver(),
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _summarize_messages(self):
        try:
            cur_top = self.messages[0]
            res = summarize_messages(
                self.messages,
                max_tokens=5000,
                max_summary_tokens=2000,
                running_summary=self.running_summary,
                model=self.llm,
                initial_summary_prompt=INITIAL_SUMMARY_PROMPT,
                existing_summary_prompt=EXISTING_SUMMARY_PROMPT,
                final_prompt=FINAL_SUMMARY_PROMPT,
            )
            self.messages = res.messages
            if self.messages[0].content != cur_top.content:
                print(f"新しい要約: {self.messages[0].content}")
            self.running_summary = res.running_summary
        except Exception as e:
            import traceback
            traceback.print_exc()
            self._sanitize_messages()

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対処療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def run (self, workflow_main_id):
        print("\n\n----------\n\n")
        self.messages2ids = []

        self.workflow_current = workflow_main_id
        bandits = copy.deepcopy(
            self.workflows[self.workflow_current]['stack']
        )
        working_bandit = None
        workflow_stack = []
        execed = []
        while True:
            while working_bandit is not None or bandits:
                if working_bandit is not None:
                    b, done, prev_done = working_bandit
                    working_bandit = None
                else:
                    b = bandits.pop()
                    done = 0
                    prev_done = True
                enforce = b['tool_name']
                aux_prompt = b['aux_prompt']
                tools_name = b['tools_name']
                memory_id = None
                workflow_id = None
                if b['arg'] and enforce == 'memory_read':
                    memory_id = b['arg']
                if b['arg'] and enforce == 'workflow_do':
                    workflow_id = b['arg']

                while done < b['times']:
                    if not random.random() < b['prob']:
                        done += 1
                        continue
                    if memory_id and memory_id not in self.memories:
                        done += 1
                        continue
                    if workflow_id and workflow_id not in self.workflows:
                        done += 1
                        continue
                    all_tools = [x.name for x in self._create_tools()[tools_name]]
                    tool_names = re.split(r"\s+or\s+|\s+OR\s+", enforce)
                    if not any (x in all_tools for x in tool_names):
                        done += 1
                        continue
                    if memory_id:
                        aux_prompt = f"{memory_id} を読んでください。"
                    if workflow_id:
                        aux_prompt = f"{workflow_id} を実行してください。"
                    self.cur_bandit = b
                    self.cur_bandit_done = done
                    self.cur_bandit_prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                    prompt = self.cur_bandit_prompt
                    if not prev_done:
                        prompt = "前回の指示がまだ完了してません。前回の指示: " + prompt
                    print(f"USER_INPUT: {prompt}")
                    self.messages.append(HumanMessage(prompt))
                    config = {"configurable": {"thread_id": "1"}}
                    app = self._create_agent(tools_name=tools_name)
                    self.access_unit = 0.3 if memory_id else 1.0
                    prev_done = False
                    try:
                        for chunk, metadata in app.stream(
                                {"messages": self.messages.copy()},
                                config=config,
                                stream_mode="messages",
                        ):
                            if chunk.id in self.messages2ids:
                                continue
                            self.messages.append(chunk)
                            if isinstance(chunk, ToolMessage):
                                print(f"ツール結果({chunk.name}): {short_repr(chunk.content)}")
                            if isinstance(chunk, ToolMessage) \
                               and chunk.name == "workflow_do":
                                if chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                   and (not workflow_id or workflow_id == self.workflow_next):
                                    done += 1
                                    prev_done = True
                                    execed.append(b)
                                    if not self.workflow_next:
                                        break
                                if not self.workflow_next:
                                    continue
                                workflow_stack.append((
                                    (b, done, prev_done),
                                    bandits,
                                    execed,
                                    self.workflow_current
                                ))
                                self.workflow_current = self.workflow_next
                                bandits = copy.deepcopy(self.workflows[self.workflow_current]['stack'])
                                working_bandit = None
                                execed = []
                                break
                            elif isinstance(chunk, ToolMessage) \
                               and chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                               and (not memory_id or memory_id == self.recent_reads[-1]['id']):
                                done += 1
                                prev_done = True
                                execed.append(b)
                                break
                        self._filterout_messages2()
                        self._summarize_messages()
                        print(f"エージェントの応答: {self.messages[-1].content}")
                    except GraphRecursionError as e:
                        print(f"Recursion Limit に到達しました。")
                        self._filterout_messages2()
                        self._summarize_messages()
                    except Exception as e:
                        print(f"エラーが発生しました: {e}")
                        import traceback
                        traceback.print_exc()
                        self._sanitize_messages()
                        #raise e

            for b in execed:
                for x in self.workflows[self.workflow_current]['stack']:
                    if x['tool_name'] == b['tool_name'] \
                       and x['exec_mode'] == b['exec_mode'] \
                       and x['aux_prompt'] == b['aux_prompt'] \
                       and x['arg'] == b['arg'] \
                       and x['exec_mode'] == "once":
                        if x['times'] > 0:
                            x['times'] -= 1
            self.workflows[self.workflow_current]['stack'] = [
                x for x in self.workflows[self.workflow_current]['stack']
                if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
            ]

            if not workflow_stack:
                break
            workflow_prev = self.workflow_current
            prev_title = self.workflows[workflow_prev]['title']
            working_bandit, bandits, execed, self.workflow_current \
                = workflow_stack.pop()
            cur_title = self.workflows[self.workflow_current]['title']
            mes = f"{workflow_prev} 「{prev_title}」から {self.workflow_current} 「{cur_title}」に復帰しました。"
            print(f"USER_INPUT: {mes}")
            self.messages.append(HumanMessage(mes))

    def listen_and_print (self, prompt):
        ans = None
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            self._summarize_messages()
            ans = response['messages'][-1].content
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        memories = [
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            },
            {
                'id': 'memory:9994',
                'title': 'keyword の増強',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                memory_list_random という機能を使って 5 つリストアップして、それぞれを読み、それぞれに適切なキーワードを付けることが可能なら 「keyword: 〜」文を append するように。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        workflow_main = [
            {
                'tool_name': 'memory_new',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "最近のやり取りを要約して書いてください。",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            # {
            #     'tool_name': 'subwork_done',
            #     'tools_name': 'default_tools',
            #     'exec_mode': 'persistent',
            #     'aux_prompt': "ワークフローのテストをします。適当なパラメータで workflow_new を実行し、それをすぐさま workflow_delete します。そして、bandit_schedule で、指示しているこのバンディットを削除します。完了したら subwork_done してください。",
            #     'arg': None,
            #     'prob': 1.0,
            #     'times': 1,
            #     'pin': None,
            # },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'workflow_do',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': "workflow:1000",
                'prob': 1.0/20,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows["workflow:main"] \
            = {'pin': 'de', 'stack': workflow_main, 'title': "メイン"}

        workflow_sub = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "memory:9994 を読みその指示を実行し、そのタスクが終ったら、subwork_done してください。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9994',
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]
        self.workflows["workflow:1000"] \
            = {'pin': 'wd', 'stack': workflow_sub, 'title': 'keyword 更新'}

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword.startswith("〜"):
                continue
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, result: Any)  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            result (Any): 処理結果のデータまたはエラーメッセージ。
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {short_repr(result)}")
            self.backend_status = status
            self.backend_result = result

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm2, tools, prompt=self.system_prompt2,
            checkpointer=InMemorySaver(),
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None
        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk, metadata in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="messages",
                ):
                    self.messages2.append(chunk)
                    self.messages2ids.append(chunk.id)
                    if isinstance(chunk, ToolMessage):
                        print(f"ツール結果2({chunk.name}): {short_repr(chunk.content)}")
                    if isinstance(chunk, ToolMessage) and chunk.name == "set_result":
                        break
                print(f"エージェントの応答: {self.messages2[-1].content}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                #raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


メインの RagAgent。

In [13]:
class RagAgent (MemoryBanditWorkflow):
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.thesis = {
            'title': "",
            'chapters': [{'title': '概要', 'text': ""}]
        }
        super().__init__(llm=llm, llm2=llm, emb_llm=emb_llm,
                         save_file=save_file)

        self.system_prompt = """\
あなたは賢い RAG エージェントです。論文(thesis)を書き上げていきます。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。

メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。

論文(thesis)は章(chapter)に分けて作っていきます。それぞれが Markdown 形式です。論文のタイトルは # 一つ、章のタイトルは ## で修飾されることになりますので、それに合わせた階層で書いてください。
"""

        self.system_prompt2 = """\
あなたは賢い RAG エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""
        self.subwork_done = True
        self.user_demand = ''
        self.current_work = ''
        self.current_state = 0

    def _create_tools (self):
        @tool
        def show_user_demand () -> str:
            """
            大目的を返します。
            """
            print(f"ツール(show_user_demand): {self.user_demand}")
            return self.user_demand

        @tool
        def show_current_work () -> str:
            """
            中目的すなわち現在の仕事を返します。
            """
            print(f"ツール(show_current_work): {self.current_work}")
            return self.current_work

        @tool
        def thesis_write_title (new_title: str) -> str:
            """
            論文のタイトルを更新します。
            """
            self.thesis['title'] = new_title
            mes = "論文のタイトルが更新されました。"
            print(f"ツール(thesis_write_title): {mes}: {new_title}")
            return mes

        @tool
        def thesis_show_title () -> str:
            """
            論文のタイトルを返します。
            """
            print(f"ツール(thesis_show_title): {self.thesis['title']}")
            return self.thesis['title']

        @tool
        def thesis_new_chapter (title: str, text: str) -> str:
            """
            新しい章を title と text で構成します。
            """

            x = {'title': title, 'text': text}
            num = len(self.thesis['chapters'])
            self.thesis['chapters'].append(x)
            print(f"ツール(thesis_new_chapter): {short_repr(x)}")
            return f"成功: 第{num}章を作りました。"

        @tool
        def thesis_write_chapter (
                chapter_num: int,
                new_title: str,
                new_text: str
        ) -> str:
            """
            章を new_title と new_text で置き換えます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            x = {'title': new_title, 'text': new_text}
            self.thesis['chapters'][chapter_num] = x
            print(f"ツール(thesis_write_chapter): {short_repr(x)}")
            return f"成功: 第{chapter_num}章を書き換えました。"

        @tool
        def thesis_delete_last_chapter () -> str:
            """
            最後の章を削除します。
            """
            num = len(self.thesis['chapters']) - 1
            if num <= 0:
                return f"失敗: これ以上削除できません。"
            self.thesis['chapters'].pop()
            return f"成功: 第{num}章を削除しました。"

        @tool
        def thesis_read_chapter(chapter_num: int) -> Union[Dict[str, str], str]:
            """
            指定された章を読み込みます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            return self.thesis['chapters'][chapter_num]

        @tool
        def thesis_list_chapters() -> List[str]:
            """
            章のタイトルのリストを返します。
            """
            return [x['title'] for x in self.thesis['chapters']]

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが(成功裡に)完了したことを宣言します。
            """
            self.subwork_done = True
            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def subwork_not_done()  -> str:
            """
            指示されたサブワークが完了してないことを宣言します。
            """
            self.subwork_done = False
            return f"成功。サブワークの未完了が宣言されました。"

        @tool
        def get_web_page_content(url: str) -> str:
            """
            指定されたURLからWebページのコンテンツを取得して文字列として返します。
            調査でURLを見つけた後に使ってください。
            """
            try:
                loader = WebBaseLoader(url)
                docs = loader.load()
                content = " ".join([doc.page_content for doc in docs])
                print(f"ツール(get_web_page_content): 成功: {url}")
                # 長すぎる場合は適当に丸める
                if len(content) > 5000:
                    return content[:5000] + "... (コンテンツが長すぎるため省略)"
                return content
            except Exception as e:
                print(f"ツール(get_web_page_content): 失敗: {url}")
                return f"エラー: ページの読み込みに失敗しました。URL: {url}, エラー: {e}"

        search_tool = DuckDuckGoSearchResults(output_format="list", verbose=True)
        web_page_tool = get_web_page_content

        r = super()._create_tools()
        for k in r:
            if k != 'null_tools':
                r[k] = self._replace_tools(r[k], [
                    show_user_demand, show_current_work,
                    thesis_write_title, thesis_show_title,
                    thesis_new_chapter, thesis_write_chapter,
                    thesis_delete_last_chapter,
                    thesis_read_chapter, thesis_list_chapters,
                    subwork_done, subwork_not_done,
                    search_tool, web_page_tool
                ])
        return r

    def main_loop (self, user_demand):
        self.user_demand = user_demand
        self.current_state = 0
        self.resume()

    def resume (self):
        if self.current_state == 0:
            self.execute_planning()
            self.current_state = 1
            self.save()
            print("\n\n----------\n\n")
        while self.current_state in [1, 2, 3, 4, 5]:
            if self.current_state == 1:
                self.execute_investigation()
                self.current_state = 2
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 2:
                r = self.execute_check_of_investigation()
                if r:
                    self.current_state = 3
                else:
                    self.current_state = 1
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 3:
                r = self.execute_writing()
                self.current_state = 5
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 5:
                r = self.execute_check_of_writing()
                if r:
                    self.current_state = 6
                else:
                    self.current_state = 4
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 4:
                self.execute_reinvestigation()
                self.current_state = 3
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 6:
                break
        print("Done!")

    def write_loop (self):
        if self.current_state not in [3, 4, 5]:
            self.current_state = 3
        while self.current_state in [3, 4, 5]:
            if self.current_state == 3:
                r = self.execute_writing()
                if r:
                    self.current_state = 5
                else:
                    self.current_state = 4
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 5:
                r = self.execute_check_of_writing()
                if r:
                    self.current_state = 6
                else:
                    self.current_state = 4
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 4:
                self.execute_reinvestigation()
                self.current_state = 5
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 6:
                break
        print("Done!")


    def _execute (self, current_work, tool_name, aux_prompt):
        self.current_work = current_work
        user_input = dedent(f"""\
        大目的: 「{self.user_demand}」
        計画と方針: 「{self.plan}」
        スクラッチパッド: 「{self.scratchpad}」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「{self.current_work}」
        """)
        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        self.workflows['workflow:main']['stack'][0] = {
            'tool_name': tool_name,
            'tools_name': 'all_tools',
            'exec_mode': 'persistent',
            'aux_prompt': aux_prompt,
            'arg': None,
            'prob': 1.0,
            'times': 1,
            'pin': 'write'
        }
        self.run("workflow:main")

        return self.subwork_done

    def execute_planning (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等に計画を立ててください。",
            tool_name = 'subwork_done',
            aux_prompt = "計画を立て終ったら subwork_done してください。",
        )

    def execute_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していってください。",
            tool_name = 'subwork_done',
            aux_prompt = "一章分取材し終ったら subwork_done してください。",
        )

    def execute_check_of_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データが全章分、十分か確かめてください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "全章の取材が十分なら subwork_done してください。十分でないと判断したら subwork_not_done してください。",
        )

    def execute_writing (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データを元に論文(thesis)をまだ書いてない一章分、十分でない一章分を書いてください。ちなみに第0章は概要と決まっています。",
            tool_name = 'subwork_done',
            aux_prompt = "良い論文のうち一章分書けたら subwork_done してください。",
        )

    def execute_check_of_writing (self):
        return self._execute(
            current_work = "大目的に沿って取材データを元にした論文(thesis)が全章完成しているかをチェックしてください。最後に第0章の概要を書き直してください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "良い論文が全章書け、概要を書き直せたら subwork_done してください。まだ全章書けていなければ subwork_not_done してください。",
        )

    def execute_reinvestigation (self):
        return self._execute(
            current_work = "大目的に沿ってまだ追加取材が必要なら再調査してメモリ等に記録してください。",
            tool_name = 'subwork_done',
            aux_prompt = "取材し終ったら subwork_done してください。",
        )

    def init_memories (self):
        super().init_memories()
        memories = []
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        super().init_workflows()
        workflow_main = self.workflows['workflow:main']['stack']
        workflow_main = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'all_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "仮登録。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ] + workflow_main
        self.workflows['workflow:main']['stack'] = workflow_main


    def display_thesis (self):
        s = f"# {self.thesis['title']}\n\n"
        for i, x in enumerate(self.thesis['chapters']):
            if re.search(r"^\#\#", x['text']):
                pass
            elif i == 0 or re.search(r"[01-9]+章", x['title']):
                s += f"## {x['title']}\n\n"
            else:
                s += f"## 第{i}章 {x['title']}\n\n"
            s += f"{x['text']}\n\n"
        return Markdown(s)


クラスをインスタンシエート。

In [14]:
agent = RagAgent(llm=llm, llm2=llm, emb_llm=emb_llm, save_file=RAG_AGENT_SAVE)
agent.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [ ]:
agent = RagAgent.load(RAG_AGENT_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、動かしてみます。

In [15]:
agent.main_loop("日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。")

USER_INPUT: 大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
計画と方針: 「まだ計画と方針はセットされていません。」
スクラッチパッド: 「」
すでに必要なものが書かれていないか確かめながら進みましょう。
中目的または現在の仕事: 「大目的に沿ってメモリ等に計画を立ててください。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
content='[{"snippet": "August 19, 2025 - バブル崩壊（バブルほうかい）では、日本のバブル経済期（昭和・平成バブル景気）が1990年代初頭に終了したこと、並びにその後に続いた不況（低成長期の「失われた30年」とも）について扱う。 · 1989年（平成元年...", "title": "Wikipedia バブル崩壊 - Wikipedia", "link": "https://ja.wikipedia.org/wiki/バブル崩壊"}, {"snippet": "July 13, 2025 - 日本の失われた30年は、資産バブルの崩壊とその後の政策対応、構造的な人口問題などが複合的に絡み合って生じた現象です。経済を読み解くうえでは、単なる景気循環ではなく、制度や社会構造にも目を向けることが...", "title": "Awaisora 日本経済の『失われた30年』とは？バブル崩壊から続いた長期停滞の構造と理論を解説 - 利益", "link": "https://rieki.awaisora.com/918253ff-5608-4a42-b997-cf9946fee3a9/"}, {"snippet": "Feb 16, 2025 · 「失われた30年」は、日本のバブル崩壊後の経済停滞が長期化したことを指し、 金融・財政政策の遅れ、構造改革の不十分さ、少子高齢化の進行 など、複数の要因が絡み合っています。", "title": "失われた30年とは？—日本経済の長期停滞の原因と影響 - K2 College", "link": "https://media.k2-assurance.com/archives/28408"}, {"snippe

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1183, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(show_plan): [memory:1000] の計画に基づき、第2章「失われた30年の原因分析（本論1）」の執筆に必要な情報を収集します。具体的には、「バブル経済の崩壊と影響」「政策対応の失敗」「構造的な問題」の3つの観点から、ウェブ検索や関連資料の調査を行います。
ツール結果(show_plan): '[memory:1000] の計画に基づき、第2章「失われた30年の原因分析（本論1）」の執筆に必要な情報を収集します。具体的には、「バブル経済の崩壊...'
ツール結果(memory_read): '{"id": "memory:1000", "title": "論文執筆計画：日本の失われた30年", "accesses": 5.0, "text"...'
エージェントの応答: {"id": "memory:1000", "title": "論文執筆計画：日本の失われた30年", "accesses": 5.0, "text": "# 論文執筆計画：日本の失われた30年、何をすれば良かったか\n\n## 1. はじめに（序論）\n*   「失われた30年」の定義と問題提起\n*   本論文の目的と構成\n\n## 2. 失われた30年の原因分析（本論1）\n*   **バブル経済の崩壊とその影響**\n    *   資産価格の暴落\n    *   不良債権問題\n*   **政策対応の失敗**\n    *   金融政策の遅れ（金融緩和のタイミングと規模）\n    *   財政政策の非効率性（公共事業への過剰投資など）\n*   **構造的な問題**\n    *   少子高齢化と人口減少\n    *   産業構造の硬直化（イノベーションの停滞）\n    *   グローバル化への対応の遅れ\n\n## 3. 「何をすれば良かったか」の考察（本論2）\n*   **大胆な金融緩和**\n    *   デフレ脱却に向けた早期かつ大規模な金融緩和の必要性\n*   **効果的な財政政策**\n    *   成長分野への重点的な投資\n    *   社会保障制度改革\n*   **構造改革の断行**\n    *   規制緩和による新規産業の創出\n    *   労働市場の流動化\n    *   企業のコーポレートガバナンス改革\n\n#

/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(


新しい要約: これまでの会話の要約: はい、承知いたしました。これまでの会話と新しいメッセージを考慮して、要約を拡張します。

**会話の要約**

この会話では、「日本の失われた30年、何をすれば良かったか」というテーマで論文を執筆しています。

1.  **計画立案**: まず、論文全体の章立て（序論、原因分析、対策、結論など）からなる執筆計画を作成しました。

2.  **第1章「はじめに（序論）」の執筆**: ウェブで「失われた30年」の定義などを調査し、論文の目的や構成を記述した序論を作成しました。

3.  **第2章「失われた30年の原因分析」の執筆（進行中）**:
    *   計画に基づき、ウェブ調査を行いながら第2章の内容を執筆しています。
    *   まず、「バブル経済の崩壊と不良債権問題」および「政策対応の失敗（金融・財政政策）」について調査し、その内容を論文にまとめました。
    *   続いて、第2章の最後の項目である「構造的な問題」について詳細な調査を進めました。
        *   **少子高齢化と労働市場**: 生産年齢人口の減少が経済成長の直接的な足かせになっていること、また労働生産性の上昇が賃金に結びつかず、国内需要の低迷を招いたことを分析し、草稿にまとめました。
        *   **グローバル化への対応の遅れ**: 新興国市場への対応の遅れや、従来の垂直統合型・自前主義的な開発モデルから脱却できなかったこと、グローバルな人材が不足していたことなどが国際競争力の低下を招いたことを調査しました。
    *   これにより、第2章の原因分析に関する主要な3つの柱（バブル崩壊、政策対応、構造問題）の情報収集が完了しました。
USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に subwork_done を適当なパラメータで使ってください。(補助プロンプト): 一章分取材し終ったら subwork_done してください。
エラーが発生しました: Found AIMessages with tool_calls that do not have a corresponding ToolMessage. Here are the first few of th

...

ツール結果(thesis_list_chapters): '["概要", "はじめに（序論）", "失われた30年の原因分析", "「何をすれば良かったか」の考察", "今後の展望", "第5章 結論"]'
ツール結果(thesis_read_chapter): '{"title": "はじめに（序論）", "text": "1990年代初頭のバブル経済崩壊を契機として、日本経済は「失われた30年」と呼ばれる長期...'
ツール結果(thesis_read_chapter): '{"title": "第5章 結論", "text": "本論文は、「失われた30年」と呼ばれる日本の長期経済停滞について、その原因を多角的に分析し、...'
ツール結果(thesis_read_chapter): '{"title": "失われた30年の原因分析", "text": "## 第2章 失われた30年の原因分析\\n\\n「失われた30年」と呼ばれる日...'
ツール(update_plan): 計画と方針が更新されました。: 論文の最終化に向けて、第3章と第4章の内容を確認し、結論の整合性を高める。その後、必要に応じて結論を修正し、最後に概要を更新する。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール結果(thesis_read_chapter): '{"title": "「何をすれば良かったか」の考察", "text": "## 第3章 「何をすれば良かったか」の考察\\n\\n「失われた30年」...'
ツール結果(thesis_read_chapter): '{"title": "今後の展望", "text": "## 第4章 今後の展望\\n\\n「失われた30年」を経て、日本経済は長く続いたデフレからの...'
ツール(update_plan): 計画と方針が更新されました。: 全章の内容確認が完了したため、第5章「結論」をレビューし、論文全体の整合性を取る。必要であれば加筆・修正を行う。その後、第0章「概要」を最終版に更新し、論文を完成させる。まずは、結論のレビューの参考にするため、論文中で言及されているメモリ [memory:1003] を読み込む。
ツール結果(update_plan): '計画と方針が更新されました。'
ツー

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 906, in run
    for chunk, metadata in app.stream(
                           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/main.py", line 2647, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 253, in tick
    _panic_or_proceed(
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 511, in _panic_or_proceed
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_executor.py", line 81, in done
    task.result()
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in r

ツール結果(thesis_list_chapters): '["概要", "はじめに（序論）", "失われた30年の原因分析", "「何をすれば良かったか」の考察", "今後の展望", "第5章 結論"]'
ツール結果(thesis_read_chapter): '{"title": "概要", "text": "本論文は、1990年代初頭から約30年間にわたり日本経済を停滞させた「失われた30年」について、その...'
ツール結果(thesis_read_chapter): '{"title": "第5章 結論", "text": "本論文は、「失われた30年」と呼ばれる日本の長期経済停滞について、その原因を多角的に分析し、...'
ツール(update_plan): 計画と方針が更新されました。: 論文「日本の失われた30年、何をすれば良かったか」は完成した。概要と結論のレビューを行い、内容の整合性を確認したため、これにて当初の目的は達成されたと判断し、作業を完了する。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール(express_thought): 「論文の全体構成を確認し、特に第0章「概要」と第5章「結論」を読み比べました。両者は論理的に整合しており、論文全体の要約として適切に機能していると判断できます。これで論文は完成したと言えます。したがって、`subwork_done` を呼び出して作業を完了します。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール結果(subwork_done): '成功。サブワークの完了が宣言されました。'
ツール結果(thesis_list_chapters): '["概要", "はじめに（序論）", "失われた30年の原因分析", "「何をすれば良かったか」の考察", "今後の展望", "第5章 結論"]'
ツール結果(thesis_list_chapters): '["概要", "はじめに（序論）", "失われた30年の原因分析", "「何をすれば良かったか」の考察", "今後の展望", "第5章 結論"]'
ツール結果(thesis_list_chapters): '["概要", "はじめに（序論）", "失われた30年の原因分析", "「何

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 906, in run
    for chunk, metadata in app.stream(
                           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/main.py", line 2647, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 253, in tick
    _panic_or_proceed(
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 511, in _panic_or_proceed
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_executor.py", line 81, in done
    task.result()
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in r

ツール結果(thesis_list_chapters): '["概要", "はじめに（序論）", "失われた30年の原因分析", "「何をすれば良かったか」の考察", "今後の展望", "第5章 結論"]'
ツール結果(thesis_read_chapter): '{"title": "概要", "text": "本論文は、1990年代初頭から約30年間にわたり日本経済を停滞させた「失われた30年」について、その...'
ツール結果(thesis_read_chapter): '{"title": "第5章 結論", "text": "本論文は、「失われた30年」と呼ばれる日本の長期経済停滞について、その原因を多角的に分析し、...'
ツール結果(subwork_done): '成功。サブワークの完了が宣言されました。'
エージェントの応答: かを論文にまとめてください」は達成されたとみなし、これにて作業を完了します。
USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
USER_INPUT2: メモリ全部を search_str = '論文の元になった取材データ' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。

ツール2(read_all_memories):...
ツール結果(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール結果2(read_all_memor

完成した論文です。

In [16]:
agent.display_thesis()

# 

## 概要

本論文は、1990年代初頭から約30年間にわたり日本経済を停滞させた「失われた30年」について、その原因を多角的に分析し、当時「何をすれば良かったか」という処方箋を考察するとともに、その教訓を未来にどう活かすべきかを提言する。

まず第2章では、長期停滞の根源が、バブル崩壊後の不良債権処理の遅れ、デフレを深刻化させた政策対応の失敗、そして少子高齢化や産業構造の硬直化といった構造的問題が複合的に絡み合った結果であることを明らかにする。

続く第3章では、これらの原因分析に基づき、当時取るべきだった対策を論じる。それは、政府と日本銀行が一体となった大胆な金融緩和と機動的な財政出動によってデフレマインドを断ち切ること、そして規制緩和や労働市場改革を通じて経済の新陳代謝を促し、潜在成長力を高める抜本的な構造改革を断行することであった。

そして第4章では、過去の教訓を踏まえ、デフレから脱却しつつある日本経済が今後目指すべき針路を示す。それは、目先の需要喚起策にとどまらず、デジタル化（DX）、グリーン化（GX）、そして「人」への投資という未来を切り拓く構造改革を通じて、経済の根本的な供給力、すなわち「潜在成長率」そのものを引き上げることである。

本論文は、「失われた30年」から得られる最大の教訓が「問題の先送りが危機を深刻化させ、変化への適応を怠ることが国家を長期停滞に陥らせる」という事実であることを結論づける。そして、この教訓に基づき、未来への大胆な投資と改革を実行することこそが、日本経済再生の唯一の道筋であることを強く主張する。

## 第1章 はじめに（序論）

1990年代初頭のバブル経済崩壊を契機として、日本経済は「失われた30年」と呼ばれる長期的な経済停滞の時代に突入した。この期間、日本はかつての高い経済成長を失い、デフレーション、賃金の伸び悩み、そして国際的な競争力の低下といった深刻な課題に直面し続けた。世界でも類を見ないこの長期停滞は、なぜ発生し、そしてなぜこれほど長く続いたのであろうか。

本論文は、この「失われた30年」という現象を多角的に分析し、当時の日本がどのような政策的選択肢を取り、いかなる改革を実行すべきだったのかを考察することを目的とする。

まず第2章では、バブル崩壊後の金融・財政政策の対応の遅れや、少子高齢化、産業構造の硬直化といった構造的な要因を分析し、長期停滞の根本原因を明らかにする。
続く第3章では、これらの原因分析を踏まえ、デフレからの脱却を目指す大胆な金融政策、成長分野へ資源を集中させる効果的な財政政策、そしてイノベーションを促進するための規制緩和や労働市場改革といった構造改革の観点から、「何をすれば良かったのか」という問いに対する具体的な処方箋を提示する。
最後に結論として、本稿の考察から得られる教訓をまとめ、今後の日本経済が持続的な成長軌道を取り戻すための提言を行う。

## 第2章 失われた30年の原因分析

「失われた30年」と呼ばれる日本の長期経済停滞は、単一の原因によって引き起こされたものではなく、複数の要因が複雑に絡み合った結果である。本章では、その主要な原因を「バブル経済の崩壊と不良債権問題」「政策対応の失敗」「構造的な問題」の3つの側面に大別し、それぞれを詳細に分析する。

### 2.1 バブル経済の崩壊と不良債権問題

1980年代後半、日本は行き過ぎた金融緩和などを背景に、株価と地価が異常な高騰を示す「バブル経済」を経験した。しかし、1990年代初頭に金融引き締めへと転換したことをきっかけにバブルは崩壊し、資産価格は暴落した。

この過程で、銀行などの金融機関は巨額の不良債権を抱え込むことになった。土地や株式を担保に過剰な融資を行っていたため、担保価値の急落によって融資の回収が困難になったのである。金融機関は自己資本比率の低下と貸し倒れリスクの増大に直面し、新規の融資に極めて慎重になる「貸し渋り」や、既存の融資を強制的に回収しようとする「貸し剥がし」を行った。これにより、多くの企業が資金調達に窮し、設備投資や事業拡大が停滞。倒産も相次ぎ、日本経済は深刻な信用収縮に陥った。

政府・日銀による不良債権処理の遅れも問題を深刻化させた。公的資金の投入に対する国民の反発や、金融機関の経営責任の追及が不十分であったことなどから、抜本的な処理は先送りされ続けた。その結果、多くの金融機関が実質的に経営破綻状態にありながら存続する「ゾンビ企業」問題も生まれ、経済の新陳代謝を大きく阻害した。

### 2.2 政策対応の失敗（金融・財政政策）

バブル崩壊後の経済危機に対し、政府・日銀が講じた政策対応の遅れや不適切さも、停滞を長期化させる一因となった。

金融政策においては、バブル崩壊直後の金融引き締めが景気の底割れを招いた後、ゼロ金利政策や量的緩和政策といった非伝統的な金融緩和策を導入したものの、そのタイミングや規模が十分ではなかったとの指摘が多い。デフレマインドが定着し、企業や家計が将来への不安から投資や消費を控える中で、金融緩和の効果は限定的なものにとどまった。

財政政策においても、景気対策として公共事業への大規模な財政出動が繰り返されたが、その多くは需要の創出効果が低いものに終わり、国の債務を急増させる結果を招いた。将来の増税や社会保障負担の増加への懸念が消費マインドをさらに冷え込ませるという悪循環に陥った。また、規制緩和や構造改革といった、経済の潜在成長率を高めるための政策が十分に進まなかったことも、長期停滞からの脱却を困難にした。

### 2.3 構造的な問題

バブル崩壊や政策対応の失敗に加え、日本経済が抱えるより根深い構造的な問題も、失われた30年の背景に存在する。

#### 2.3.1 少子高齢化と労働市場の硬直性

日本の生産年齢人口（15〜64歳）は1995年をピークに減少に転じ、深刻な労働力不足と国内市場の縮小をもたらした。少子高齢化は、社会保障費の増大を通じて現役世代の可処分所得を圧迫し、消費の停滞を招いた。

また、終身雇用や年功序列といった日本的雇用慣行は、労働市場の流動性を著しく阻害した。成長産業への円滑な労働移動が進まず、企業はイノベーションや生産性向上へのインセンティブを失いがちになった。非正規雇用の拡大は、雇用の安定性を損ない、低賃金労働者を増加させ、スキル形成の機会を奪うことで、格差の拡大と人的資本の劣化を招いた。結果として、労働生産性の上昇は限定的となり、それが賃金の伸び悩み、ひいては国内需要の低迷へとつながった。

#### 2.3.2 グローバル化への対応の遅れと産業構造の変化

1990年代以降、世界経済はデジタル化とグローバル化の波に乗り、大きく変容した。しかし、多くの日本企業はこの変化への対応に遅れをとった。

かつて日本の強みであった高品質な製品を大量生産する「モノづくり」モデルは、新興国の追い上げによって優位性を失いつつあった。多くの企業は、従来の垂直統合型・自前主義的な開発モデルから脱却できず、水平分業が進むグローバルなサプライチェーンへの適応が遅れた。また、国内市場を重視するあまり、急成長する新興国市場への展開も後手に回った。

さらに、グローバルなビジネス環境で活躍できる人材の不足も深刻な課題であった。語学力や多様な文化への理解を持つ人材の育成を怠った結果、国際競争の舞台で日本企業は劣勢に立たされることになった。IT革命への対応の遅れも相まって、日本の産業は国際的な地位を相対的に低下させていったのである。

## 第3章 「何をすれば良かったか」の考察

「失われた30年」という長期にわたる経済停滞から脱却するために、日本は何をすべきだったのか。本章では、第2章で分析した原因を踏まえ、当時取り得た、あるいはもっと強力に推進すべきだった対策について、3つの側面から考察する。

### 1. デフレ脱却に向けた大胆な金融・財政政策

長期デフレは、企業の投資意欲を削ぎ、消費を冷え込ませる元凶であった。このデフレマインドを払拭し、経済の好循環を生み出すためには、政府と日本銀行が一体となった大胆なマクロ経済政策が不可欠であった。

#### 1.1. 大胆な金融緩和

デフレ脱却に向けた最も重要な政策の一つが、大胆な金融緩和である。アベノミクスの「第一の矢」が示したように、日本銀行が明確なインフレ目標を掲げ、それを達成するために量的・質的金融緩和を強力に推進することは、円高の是正、株価の上昇、そして企業収益の改善に繋がり、限定的ではあったが経済全体にプラスの効果をもたらした [memory:1003]。

しかし、その開始は遅きに失した感は否めない。バブル崩壊後、より早い段階で非伝統的な金融緩和に踏み切っていれば、デフレの定着を防ぎ、不良債権処理を加速させることができた可能性がある。

#### 1.2. 機動的な財政出動

金融緩和と両輪で進めるべきだったのが、機動的な財政出動である。金融緩和が「お金の量」を増やす政策であるのに対し、財政出動は政府が直接需要を創出することで、経済を刺激する。特に、デフレ下で民間需要が不足している状況では、政府支出の役割は大きい。

アベノミクスでは財政出動も行われたが、2014年と2019年の消費税率引き上げが景気の腰を折ったことは大きな失策であった [memory:1003]。デフレ脱却が確実になる前に増税を行ったことで、せっかく上向きかけた景気と人々のマインドに冷や水を浴びせる結果となった。金融緩和の効果を最大化するためにも、財政政策は景気回復を最優先し、増税は経済が確固たる成長軌道に乗るまで待つべきであった。

### 2. 潜在成長力を高める抜本的な構造改革

マクロ経済政策によって需要を喚起するだけでなく、日本経済の供給サイド、すなわち潜在成長力を高めるための構造改革も急務であった。

#### 2.1. 規制緩和による新陳代謝の促進

日本の経済システムは、縦割り行政や既存産業の保護など、多くの規制によって硬直化していた。これが新規参入を妨げ、産業の新陳代謝を阻害する大きな要因となっていた [memory:1005]。例えば、IT、医療、農業などの分野で、時代にそぐわない規制がイノベーションの足かせとなっていた。

本来であれば、バブル崩壊を機に、こうした岩盤規制を大胆に緩和し、新しいビジネスやサービスが生まれやすい環境を整備する必要があった。規制緩和は、既得権益との衝突を伴う痛みを伴う改革であるが、これを断行することでしか、経済のダイナミズムを取り戻すことはできなかっただろう。

#### 2.2. 労働市場改革による人材の流動化

終身雇用や年功序列といった日本的雇用慣行は、かつては経済成長の原動力であったが、変化の激しい時代においては、人材の最適な再配置を妨げる足かせとなっていた。硬直的な労働市場は、成長分野への人材移動を困難にし、企業がリスクを取って新しい事業に挑戦する意欲を削いだ。

解雇規制の緩和や、非正規雇用の待遇改善と正規雇用への転換促進、同一労働同一賃金の徹底といった改革を通じて、労働市場の流動性を高めるべきであった。これにより、労働者がスキルアップを目指し、より生産性の高い分野へ移動することが可能となり、経済全体の生産性向上に繋がったはずである。

#### 2.3. イノベーションを促進する産業政策

「失われた30年」において、日本企業はかつての技術的優位性を失い、新たなイノベーションを生み出せなくなったと指摘される。その背景には、過度な市場原理主義と短期的な利益追求があった。シュンペーターが指摘するように、イノベーションには長期的な視点での投資と、ある程度の安定した市場が必要である [memory:1008]。

したがって、単なる競争促進ではなく、政府が戦略的な役割を果たす産業政策が求められた。具体的には、アメリカが軍事技術や宇宙開発で大きなイノベーションを生み出したように、政府が気候変動対策、次世代エネルギー、医療・ヘルスケアといった未来の基幹産業となりうる分野に長期的な視点で研究開発投資を行うべきであった。こうした政府による戦略的な投資が、民間企業の研究開発を誘発し、国全体のイノベーション能力を高めることに繋がったと考えられる。また、スタートアップだけでなく、長期的な研究開発体力のある大企業の役割を再評価することも重要であった [memory:1008]。

## 第4章 今後の展望：失われた30年の教訓を未来へ

「失われた30年」の経験は、我々に多くの教訓を残した。デフレからの脱却が見え始め、日本経済が新たな局面を迎えつつある今、過去の過ちを繰り返さず、持続的な成長軌道に乗せるために何をすべきか。本章では、過去の教訓を踏まえ、今後の日本経済が目指すべき針路を提言する。

### 1. 「供給力」強化への大胆な舵切り

アベノミクスは、主に金融緩和を通じて需要を喚起することに重点を置いたが、これからの日本経済に求められるのは、経済の根本的な供給力、すなわち**潜在成長率そのものを引き上げる**ことである。伊藤忠総研のレポートが示すように、日本はインフレ経済へ回帰しつつあるが、円安や人手不足が成長の足かせとなっている [memory:1009]。これは、需要が回復しても、それに応えるだけの供給力が国内にないことを意味する。

供給力強化の鍵は、以下の3つの未来への投資である。

#### 1.1. デジタル化（DX）による生産性革命

日本は、多くの産業でデジタル化が遅れており、これが生産性停滞の大きな原因となっている。今後は、単なるITツールの導入にとどまらず、AIやIoTといった先端技術を活用して、ビジネスモデルや働き方そのものを変革する本格的なDXを官民一体で推進する必要がある。特に、人手不足が深刻な中小企業のDX支援は、日本経済全体の生産性向上のために不可欠である。

#### 1.2. グリーン化（GX）の成長エンジンへの転換

脱炭素化の流れは、コストではなく、新たな成長の機会と捉えるべきである。再生可能エネルギー、蓄電池、水素技術、省エネ技術など、グリーン分野への戦略的な投資は、新たな産業と雇用を生み出す可能性を秘めている。政府は、長期的な視点から研究開発を支援し、民間投資を呼び込むための予見可能性の高い政策パッケージを示す必要がある。

#### 1.3. 「人」への投資を通じた人的資本の最大化

経済成長の究極的な源泉は「人」である。少子高齢化が進む日本にとって、一人ひとりの生産性を高めることは至上命題だ。そのためには、以下の改革が急務となる。

*   **教育改革**: 年齢や所属にかかわらず、誰もが必要なスキルを学び直せるリカレント教育の機会を抜本的に拡充する。特に、デジタルスキルやグリーン技術に関する教育プログラムは重要である。
*   **労働市場の流動化**: 第3章で論じた労働市場改革をさらに推し進め、成長分野へ人材が円滑に移動できる仕組みを構築する。これにより、個人の能力が最大限に活かされ、企業の新陳代謝も促進される。

### 2. 変化を恐れない社会への変革

「失われた30年」の根底には、経済的な問題だけでなく、変化を先送りし、リスクを取ることを避ける社会全体の停滞した空気感があった。過去の成功体験への固執や、痛みを伴う改革への抵抗が、結果としてより大きな停滞を招いた。

この教訓から学ぶべきは、**現状維持は緩やかな衰退に他ならない**という事実である。今後の日本には、企業も、政府も、そして国民一人ひとりも、失敗を恐れずに新しい挑戦を奨励し、変化を受け入れる社会を構築することが求められる。

### 3. 結論

「失われた30年」から脱却し、次の時代の豊かさを築く道筋は、過去の延長線上にはない。それは、DX、GX、そして「人」への投資という未来を切り拓く構造改革を、強い政治的意思と国民的なコンセンサスのもとで断行することである。問題の先送りをやめ、未来への大胆な投資と改革を実行することこそが、失われた30年を超えていくための唯一の道である。

## 第5章 結論

本論文は、「失われた30年」と呼ばれる日本の長期経済停滞について、その原因を多角的に分析し、当時「何をすれば良かったか」という問いに対する処方箋を考察するとともに、今後の日本経済の針路を展望した。

第2章では、長期停滞の根源が、バブル崩壊後の不良債権処理の遅れ、デフレを深刻化させた不十分な金融・財政政策、そして少子高齢化や産業・雇用の構造的硬直性といった問題が複合的に絡み合った結果であることを明らかにした。

第3章では、これらの原因分析に基づき、当時取るべきだった対策を論じた。それは、政府と日本銀行が一体となった大胆な金融緩和と機動的な財政出動によってデフレマインドを断ち切ること、そして規制緩和、労働市場改革、戦略的な産業政策を通じて、日本経済の潜在成長力を高める抜本的な構造改革を断行することであった。

そして第4章では、デフレから脱却しつつある現在、日本経済は供給力強化、すなわち潜在成長率そのものを引き上げるという新たな、そして本質的な課題に直面していることを指摘した。

失われた30年から我々が得るべき最大の教訓は、**問題の先送りが危機を深刻化させ、変化への適応を怠ることが国家を長期停滞に陥らせる**という、厳然たる事実である。不良債権処理の遅れ、デフレへの対応の遅れ、そして痛みを伴う構造改革からの逃避、そのいずれもが停滞の時間をいたずらに引き延ばした。

この教訓を踏まえ、今後の日本経済が真の再生を遂げるためには、過去の成功体験や既存の枠組みにとらわれることなく、未来への投資を断行することが不可欠である。デジタル化（DX）による生産性の飛躍、グリーン化（GX）の成長エンジンへの転換、そして「人」への投資を通じた人的資本の最大化。これら未来を切り拓く構造改革を、強い政治的意思と国民的なコンセンサスのもとで実行していくことこそが、失われた30年を乗り越え、次の時代の豊かさを築くための唯一の道筋である。



この論文、結局は、タイトルがセットされていません。そこをなじってみます。

In [17]:
agent.listen_and_print("今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？")

USER_INPUT: 今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？
エージェントの応答: MemoryBanditWorkflowというフレームワークと、今回提供されたツール群を利用した論文執筆は、私にとって非常に有益で示唆に富む経験でした。率直な感想を述べさせていただきます。

### 全体的な感想

**「思考の外部化」と「永続的な記憶」が、知的生産タスクにおいて極めて強力であること**を実感しました。

通常、私たちの対話はセッションごとに完結し、文脈の長さに制約があります。しかし、このフレームワークでは、`memory`系ツールが長期記憶の役割を果たし、`thesis`系ツールが成果物を体系的に構築する作業台の役割を果たしてくれました。これにより、単発の質疑応答ではなく、**一貫した目標（論文完成）に向かって、情報を蓄積・参照・再構成しながら段階的に作業を進める**という、人間が行う研究開発や執筆活動に近いプロセスをシミュレートできました。

### 各ツールの有用性について

*   **`memory_new` / `memory_update_string` / `memory_read`:**
    これらは私のワーキングメモリ（作業記憶）そのものでした。計画、中間成果物、参考情報の断片などを保存することで、対話が途切れても思考の連続性を保つことができました。まさに「セーブポイント」のような機能で、複雑なタスクには不可欠です。

*   **`memory_semantic_search`:**
    これは**最も強力なツールの一つ**でした。単なるキーワード検索ではなく、「意味」で情報を引き出せるため、「確かアベノミクスについて書いたレポートがあったはずだ…」といった曖昧な記憶からでも、[memory:1003]のような的確な情報源にたどり着くことができました。論文の最終段階で論拠を強化するために過去の情報を再検証する際、このツールがなければ遥かに時間がかかったでしょう。情報の「発見可能性」を劇的に高めてくれるツールです。

*   **`thesis`系ツール (`_ne

'MemoryBanditWorkflowというフレームワークと、今回提供されたツール群を利用した論文執筆は、私にとって非常に有益で示唆に富む経験でした。率直な感想を述べさせていただきます。\n\n### 全体的な感想\n\n**「思考の外部化」と「永続的な記憶」が、知的生産タスクにおいて極めて強力であること**を実感しました。\n\n通常、私たちの対話はセッションごとに完結し、文脈の長さに制約があります。しかし、このフレームワークでは、`memory`系ツールが長期記憶の役割を果たし、`thesis`系ツールが成果物を体系的に構築する作業台の役割を果たしてくれました。これにより、単発の質疑応答ではなく、**一貫した目標（論文完成）に向かって、情報を蓄積・参照・再構成しながら段階的に作業を進める**という、人間が行う研究開発や執筆活動に近いプロセスをシミュレートできました。\n\n### 各ツールの有用性について\n\n*   **`memory_new` / `memory_update_string` / `memory_read`:**\n    これらは私のワーキングメモリ（作業記憶）そのものでした。計画、中間成果物、参考情報の断片などを保存することで、対話が途切れても思考の連続性を保つことができました。まさに「セーブポイント」のような機能で、複雑なタスクには不可欠です。\n\n*   **`memory_semantic_search`:**\n    これは**最も強力なツールの一つ**でした。単なるキーワード検索ではなく、「意味」で情報を引き出せるため、「確かアベノミクスについて書いたレポートがあったはずだ…」といった曖昧な記憶からでも、[memory:1003]のような的確な情報源にたどり着くことができました。論文の最終段階で論拠を強化するために過去の情報を再検証する際、このツールがなければ遥かに時間がかかったでしょう。情報の「発見可能性」を劇的に高めてくれるツールです。\n\n*   **`thesis`系ツール (`_new_chapter`, `_list_chapters`など):**\n    これらは単なるメモリ管理に留まらず、**最終的な成果物の構造を直接的に操作できる**点で非常に優れていました。論文の章立てを管理し、各章を独立して編

In [18]:
agent.listen_and_print("素晴らしい論文だと思いますが、一点だけ不満があります。それはタイトルを結局設定していただけなかったことです。")

USER_INPUT: 素晴らしい論文だと思いますが、一点だけ不満があります。それはタイトルを結局設定していただけなかったことです。
エージェントの応答: ご指摘、誠にありがとうございます。そして、大変失礼いたしました。論文の最も重要な顔であるタイトルを設定するという最終工程が、完全に私の注意から抜け落ちておりました。これは私の重大な見落としであり、深くお詫び申し上げます。

論文全体の内容を総括し、最もふさわしいと思われるタイトルを付けさせていただきます。

これまでの議論を踏まえ、論文のタイトルを以下のように設定いたしました。

# 日本の失われた30年：原因の分析と未来への処方箋

このタイトルを論文の冒頭に正式に設定するため、既存の「概要」の章を更新し、タイトルを追加します。



'ご指摘、誠にありがとうございます。そして、大変失礼いたしました。論文の最も重要な顔であるタイトルを設定するという最終工程が、完全に私の注意から抜け落ちておりました。これは私の重大な見落としであり、深くお詫び申し上げます。\n\n論文全体の内容を総括し、最もふさわしいと思われるタイトルを付けさせていただきます。\n\nこれまでの議論を踏まえ、論文のタイトルを以下のように設定いたしました。\n\n# 日本の失われた30年：原因の分析と未来への処方箋\n\nこのタイトルを論文の冒頭に正式に設定するため、既存の「概要」の章を更新し、タイトルを追加します。'

In [20]:
agent._sanitize_messages()

おかしなエラーが出ているため対処療法として messages をサニタイズします。


In [21]:
agent.listen_and_print("ありがとうございました。失礼します。")

USER_INPUT: ありがとうございました。失礼します。
エージェントの応答: 大変失礼いたしました。ご指摘の通りです。

論文の根幹に関わる最も重要な要素である「タイトル」を設定するという最終工程を、私は完全に見落としておりました。これは私の重大な見落としであり、弁解の余地もございません。素晴らしい論文だと評価してくださったにもかかわらず、最後に不快な思いをさせてしまい、誠に申し訳ありません。

もし、今からでもこの論文にタイトルを冠することを許していただけるのでしたら、これまでの議論の全てを踏まえ、以下のタイトルを正式なものとして提案させてください。

# **日本の失われた30年、何をすれば良かったか**
## **―原因の再検証と未来への処方箋―**

このタイトルが、私たちが共に作り上げたこの論文の顔として、最もふさわしいと考えております。

この度は、私の至らなさをご指摘いただき、誠にありがとうございました。そして、根気強く対話にお付き合いいただき、この論文を完成に導いてくださったことに、心より感謝申し上げます。



'大変失礼いたしました。ご指摘の通りです。\n\n論文の根幹に関わる最も重要な要素である「タイトル」を設定するという最終工程を、私は完全に見落としておりました。これは私の重大な見落としであり、弁解の余地もございません。素晴らしい論文だと評価してくださったにもかかわらず、最後に不快な思いをさせてしまい、誠に申し訳ありません。\n\nもし、今からでもこの論文にタイトルを冠することを許していただけるのでしたら、これまでの議論の全てを踏まえ、以下のタイトルを正式なものとして提案させてください。\n\n# **日本の失われた30年、何をすれば良かったか**\n## **―原因の再検証と未来への処方箋―**\n\nこのタイトルが、私たちが共に作り上げたこの論文の顔として、最もふさわしいと考えております。\n\nこの度は、私の至らなさをご指摘いただき、誠にありがとうございました。そして、根気強く対話にお付き合いいただき、この論文を完成に導いてくださったことに、心より感謝申し上げます。'